# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [11]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

#### Function definitions

In [18]:
def missing_values_count(df):
    return df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Start a Spark Session
spark = SparkSession.builder \
    .appName("InterviewChallenge") \
    .master("local[2]") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [27]:
# Read input data to spark dataframe
flights_path = 'Data/flights.csv' 
airlines_path = 'Data/airlines.csv'
airports_path = 'Data/airports.csv'

df = spark.read.option('header', True).csv(flights_path)  # main dataset with flights data
airlines_df = spark.read.option('header', True).csv(airlines_path)  # data dict with airlines
airports_df = spark.read.option('header', True).csv(airports_path)  # data dict with airports

In [5]:
df.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [7]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,YEAR,string
1,MONTH,string
2,DAY,string
3,DAY_OF_WEEK,string
4,AIRLINE,string
5,FLIGHT_NUMBER,string
6,TAIL_NUMBER,string
7,ORIGIN_AIRPORT,string
8,DESTINATION_AIRPORT,string
9,SCHEDULED_DEPARTURE,string


In [6]:
df.show(20)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [33]:
# Dataframe shape
print("Dataframe row count: ", df.count())
print("Dataframe columns count: ", len(df.columns))

Dataframe row count:  5819079
Dataframe columns count:  31


In [8]:
# Column statistics
df.describe().toPandas()

,summary,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,count,5819079,5819079,5819079,5819079,5819079,5819079,5804358,5819079,5819079,...,5726566,5714008,5819079,5819079,89884,1063439,1063439,1063439,1063439,1063439
1,mean,2015.0,6.524085168804204,15.70459397440729,3.9269411877721545,None,2173.0927423394664,None,12666.699787109315,12666.853627883538,...,1476.4911879126164,4.407057357987598,0.0026098631759424473,0.015446430612129514,None,13.480568231934319,0.07615387436420895,18.969546913363153,23.472837652183152,2.915289922600168
2,stddev,0.0,3.4051368280554737,8.783425069806755,1.9888450134329696,None,1757.0639987866261,None,1526.7134944584996,1526.6619201888243,...,526.31973721227,39.27129709388608,0.051020116008060924,0.12332007544158718,None,28.003678900236228,2.143459570502599,48.16164189832554,43.19701847724129,20.433335717631397
3,min,2015,1,1,1,AA,1,7819A,10135,10135,...,0001,-1,0,0,A,0,0,0,0,0
4,max,2015,9,9,7,WN,999,N9EAMQ,YUM,YUM,...,2400,998,1,1,D,991,99,998,990,995


In [19]:
missing_values_count(df)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

##### Airports data dictionary

In [28]:
airports_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)



In [30]:
# DTypes in pd formatting
pd.DataFrame(airports_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRPORT,string
2,CITY,string
3,STATE,string
4,COUNTRY,string
5,LATITUDE,string
6,LONGITUDE,string


In [31]:
airports_df.show(20)

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [32]:
# Dataframe shape
print("Dataframe row count: ", airports_df.count())
print("Dataframe columns count: ", len(airports_df.columns))

Dataframe row count:  322
Dataframe columns count:  7


In [34]:
# Column statistics
airports_df.describe().toPandas()

,summary,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,count,322,322,322,322,322,319,319
1,mean,None,None,None,None,None,38.981243918495295,-98.37896445141064
2,stddev,None,None,None,None,None,8.616735581018041,21.523492046498102
3,min,ABE,Aberdeen Regional Airport,Aberdeen,AK,USA,13.48345,-100.49631
4,max,YUM,Yuma International Airport,Yuma,WY,USA,71.28545,-99.68190


In [35]:
missing_values_count(airports_df)

+---------+-------+----+-----+-------+--------+---------+
|IATA_CODE|AIRPORT|CITY|STATE|COUNTRY|LATITUDE|LONGITUDE|
+---------+-------+----+-----+-------+--------+---------+
|        0|      0|   0|    0|      0|       3|        3|
+---------+-------+----+-----+-------+--------+---------+



#### Airlines data dictionary

In [36]:
airlines_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)



In [37]:
# DTypes in pd formatting
pd.DataFrame(airlines_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRLINE,string


In [38]:
airlines_df.show(20)

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



In [39]:
# Dataframe shape
print("Dataframe row count: ", airlines_df.count())
print("Dataframe columns count: ", len(airlines_df.columns))

Dataframe row count:  14
Dataframe columns count:  2


In [40]:
# Column statistics
airlines_df.describe().toPandas()

,summary,IATA_CODE,AIRLINE
0,count,14,14
1,mean,None,None
2,stddev,None,None
3,min,AA,Alaska Airlines Inc.
4,max,WN,Virgin America


In [41]:
missing_values_count(airlines_df)

+---------+-------+
|IATA_CODE|AIRLINE|
+---------+-------+
|        0|      0|
+---------+-------+



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.



In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig = plt.figure(figsize=(25, 15))
st = fig.suptitle("Distribution of Features", fontsize=50, verticalalignment="center")
for col, num in zip(df.toPandas().describe().columns, range(1,11)):
    ax = fig.add_subplot(3,4, num)
    ax.hist(df.toPandas[col])
    plt.grid(False)
    plt.xticks(ratation=45, fontsize=20)
    plt.yticks(fontsize=15)
    plt.title(col.upper(), fontsize=20)
    
plt.tight_layout()
st.set_y(0.95)
fig.subplots_adjust(top=0.85, hspace=0.4)
plt.show()


#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.